# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [5]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [6]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: @EikiMarcelo

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

Salvando os dados em uma planilha Excel:

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [7]:
#def de limpar palavras
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
#Excel
skol_read=pd.read_excel("skol.xlsx")

In [8]:
#Limpeza e separação por relevancia
skol_read.Treinamento.apply(cleanup)
skol_read.rename(columns={'Unnamed: 1':'relevante'},inplace=True)
#Relevante 0
skol_relevante_0=skol_read[skol_read.relevante==0]

#Relevante 1
skol_relevante_1=skol_read[skol_read.relevante==1]
print ("Len relevante 0:",len(skol_relevante_0),"",
       "Len relevante 1:",len(skol_relevante_1))

Len relevante 0: 185  Len relevante 1: 115


In [9]:
# Separando as palavras Relevancia 1
" ".join(skol_relevante_1["Treinamento"]).split()
palavras=pd.DataFrame("".join(skol_relevante_1["Treinamento"]).split())
palavras_relevancia_1=palavras[0].value_counts()

#Tabela Relativa 
palavras_relevancia_1_relativa=palavras[0].value_counts(True)

In [10]:
# Separando as palavras Relevancia 0
" ".join(skol_relevante_0["Treinamento"]).split()
palavras=pd.DataFrame("".join(skol_relevante_0["Treinamento"]).split())
palavras[0].value_counts()
palavras_relevancia_0=palavras[0].value_counts()
palavras_relevancia_0

#Tabela Relativa 
palavras_relevancia_0_relativa=palavras[0].value_counts(True)

In [11]:
print (palavras_relevancia_0_relativa)

skol                          0.040550
de                            0.034556
e                             0.023625
a                             0.022214
beats                         0.014457
o                             0.013399
eu                            0.013047
da                            0.012341
que                           0.011636
pra                           0.010578
uma                           0.010226
do                            0.009520
é                             0.009520
me                            0.008110
com                           0.007757
no                            0.007052
não                           0.006347
q                             0.006347
em                            0.005994
na                            0.005642
se                            0.005642
um                            0.005642
tu                            0.005289
bebendo                       0.004937
vai                           0.004584
mano                     

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### <font color = "red"> Lista Prob Relevância 0 (Lista_prob_0):

In [51]:
i=0
lista_frase=[]
lista_palavra = []
while i <=299:
    a=skol_read.iloc[i,0]
    lista_frase.append(a)
    x = lista_frase[i].split(" ")
    lista_palavra.append(x)
    i+=1
lista_palavra



t = 1/(len(palavras_relevancia_0_relativa)+len(palavras_relevancia_0_relativa))

i = 0
produto_prob = []
while i <= 299:
    k = 0
    mult_prob = []
    while k < len(lista_palavra[i]):
        palavra = lista_palavra[i][k]
        if palavra not in palavras_relevancia_0_relativa:
            prob = t
        else:
            prob = palavras_relevancia_0_relativa[palavra] + t
        mult_prob.append(prob)
        k+=1
    #Aqui ja tem lista da probabilidade das palavras
    #print(mult_prob)
    p = 0
    for e in mult_prob:
        if p == 0:
            p = e
        else:
            p = p*e
    produto = p
    produto_prob.append(produto)
    i+=1
Lista_prob_0 = produto_prob

### <font color = "red" > Lista Prob Relevância 1 (Lista_prob_1):

In [54]:
i=0
lista_frase=[]
lista_palavra = []
while i <=299:
    a=skol_read.iloc[i,0]
    lista_frase.append(a)
    x = lista_frase[i].split(" ")
    lista_palavra.append(x)
    i+=1
lista_palavra



t = 1/(len(palavras_relevancia_1_relativa)+len(palavras_relevancia_1_relativa))

i = 0
produto_prob = []
while i <= 299:
    k = 0
    mult_prob = []
    while k < len(lista_palavra[i]):
        palavra = lista_palavra[i][k]
        if palavra not in palavras_relevancia_1_relativa:
            prob = t
        else:
            prob = palavras_relevancia_1_relativa[palavra] + t
        mult_prob.append(prob)
        k+=1
    #Aqui ja tem lista da probabilidade das palavras
    #print(mult_prob)
    p = 0
    for e in mult_prob:
        if p == 0:
            p = e
        else:
            p = p*e
    produto = p
    produto_prob.append(produto)
    i+=1
Lista_prob_1 = produto_prob

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**